## Grouping tickers

Ticker	Company Name	Unique Buyers	Total Purchases	Positions Represented

#### Improved grouping.

1: Instead of looping from index i to index j where j has date i plus two days, a mask is used with a date and ticker condition.

2: Instead of using a pd series for used indexs, a set() was added to by taking the .index list from the masked pd df (group_df).

3: The unique_buyers and all_titles were created by taking sets of the cols of group_df.

4: A dict was then created with the *final* numbers and then added to the list of grouped trades, and then that list was converted to a pd df.

In [ ]:


df = df.sort_values(by='transaction_date').reset_index(drop=True)
grouped = []
used_indices = set()

for idx, row in df.iterrows():
    if idx in used_indices:
        continue

    ticker = row['ticker']
    start_date = row['transaction_date']
    cutoff_date = start_date + timedelta(days=5)

    # Filter trades for this ticker within the 2-day window
    mask = (
        (df['ticker'] == ticker) &
        (df['transaction_date'] >= start_date) &
        (df['transaction_date'] <= cutoff_date)
    )
    group_df = df[mask]
    group_indices = group_df.index.tolist() 
    used_indices.update(group_indices)

    unique_buyers = set(group_df['owner_name'])
    all_titles = set(group_df['Title'])

    earliest_trade_date = group_df['trade_date'].min()

    unique_buyers_count = len(unique_buyers) 

    

    grouped.append({
        'transaction_date': start_date,
        'trade_date': earliest_trade_date,
        'ticker': ticker,
        'company_name': row['company_name'],
        'unique_buyers': unique_buyers_count,
        'total_purchases': len(group_df),
        'Title': list(all_titles),
        'owner_names': list(unique_buyers)
    })



df_by_ticker = pd.DataFrame(grouped) #note that trade_date is the earliest transaction 
#print(df_by_ticker)

def adjust_transaction_date(dt):
    if dt.hour < 9 or (dt.hour == 9 and dt.minute < 30):
        return (dt - pd.Timedelta(days=1)).date()
    else:
        return dt.date()

df_by_ticker['eff_trans_date'] = df_by_ticker['transaction_date'].apply(adjust_transaction_date)

## Downloading Ticker prices

### Retreiving ticker prices and key metrics

Binary up down for the next day 

We want to find the percent change over the following day, 5 days, 20 days (excluding weekends).

Keep a 30 day price plot maybe for predicting 

for more scope creep, using a 30 (or 60 or 90) day price plot estimate the most likley day that the stock will peak and use this as the sell date.


In [ ]:
#This was slow and trash, better to bulk download and select the 90 days after
'''
trade = df_by_ticker.iloc[1]
print(trade)
start= trade['transaction_date'].date()  - timedelta(days=91)
end = trade['transaction_date'].date() - timedelta(days=1)

ticker = yf.Ticker(trade['ticker'])
ticker_data = ticker.history(start=start, end=end)[['Open', 'High', 'Low', 'Close', 'Volume']]

def get_ticker_data_90_days(trade): #pass in row of df that incrudes the trade date and the ticker symbol
    #print(trade)
    start= trade['transaction_date'].date()  - timedelta(days=91)
    end = trade['transaction_date'].date() - timedelta(days=1)
    return yf.Ticker(trade['ticker']).history(start=start, end=end)[['Open', 'High', 'Low', 'Close', 'Volume']]


df_ticker_data = df_by_ticker.apply(lambda row: get_ticker_data_90_days(row), axis=1)
'''

#### Bulk download ticker data and then pickout the 90 day bits you need

In [ ]:
tickers = df['ticker'].unique()

# Bulk download full date range covering all needed periods
# any missing failed downloads with still have a row but poplated with NaN's
start=df['trade_date'].min() - timedelta(days=120)
end= df['transaction_date'].max() + timedelta(days=60)

downloaded_prices = yf.download(
    tickers=list(tickers),
    start=start,
    end=end
)[['Open', 'High', 'Low', 'Close', 'Volume']]

#spy_history = yf.Ticker('SPY').history(start=start, end=end)[['Open', 'High', 'Low', 'Close', 'Volume']]

C:\Users\aaron\AppData\Local\Temp\ipykernel_7484\4003368523.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  downloaded_prices = yf.download(
[**********************60%****                   ]  4808 of 7975 completedFailed to get ticker '..' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  7975 of 7975 completed

3051 Failed downloads:
['BTRS', 'SMMC', 'MIIIU', 'BIOC', 'GCP', 'TNTR', 'GLLA', 'GFGD', 'PNT', 'CVGVW', 'TRTK', 'MCEP', 'CRDAC', 'NOSYM', 'CIR', 'IMGN', 'NEP', 'BERY', 'FNHM', 'AIKI', 'KNBE', 'ABTX', 'ISBC', 'ADOC', 'VCSA', 'FSRXU', 'ZYNE', 'KURI', 'DISTU', 'VKIN', 'TETE', 'HCFT', 'SASI', 'WNDM', 'NXTD', 'CVA', 'BBLS', 'RADI', 'SGH', 'TCF', 'PSDO', 'ARYA', 'AERI', 'FLFV', 'MIC', 'CBLAQ', 'MYHI', 'BAS', 'EMBI', 'SCTY', 'CAW', 'SCHN', 'BLXX', 'MBTF', 'FIXX', 'OMBP', 'HCP', 'GBT', 'POPE', 'FLLCU', 'DBTX', 'INFN', 'CBDEF', 'DSKX', 'MSRT', 'ACGN', 'SRSA', 'NUVA', 'CVCY', 'PACE', 'CBIS', 'NP', 

In [ ]:


# Slice per row from this cached dataset
def get_ticker_data_from_cache(trade):
    prev_price = downloaded_prices.loc[trade['eff_trans_date'] - timedelta(days=100):trade['eff_trans_date'], (slice(None), trade['ticker'])]
    prev_price.columns = prev_price.columns.droplevel(1)

    future_price = downloaded_prices.loc[trade['eff_trans_date']+ timedelta(days=1) : trade['eff_trans_date'] + timedelta(days=31), (slice(None), trade['ticker'])]
    future_price.columns = future_price.columns.droplevel(1)

    row = {
        'ticker': trade['ticker'],
        'prev_prices': prev_price,
        'future_prices' : future_price
    }
    return row

df_ticker_data = df_by_ticker.apply(get_ticker_data_from_cache, axis=1, result_type='expand')

df_by_ticker['prev_prices'] = df_ticker_data['prev_prices']
df_by_ticker['future_prices'] = df_ticker_data['future_prices']


### Another ticker download method
#### Since the above method will download 10 years worth of ticker data for 150K trades.

In [ ]:
ticker_ranges = df.groupby("ticker")["trade_date"].agg(
    min_date=lambda x: x.min() - pd.DateOffset(months=4),
    max_date=lambda x: x.max() + pd.DateOffset(months=2)
).reset_index()

In [ ]:
prices = {}

for _, row in ticker_ranges.iterrows():
    t = row["ticker"]
    start, end = row["min_date"], row["max_date"]
    try:
        data = yf.download(t, start=start, end=end)[['Open','High','Low','Close','Volume']]
        prices[t] = data
    except Exception as e:
        print(f"Failed {t}: {e}")

C:\Users\aaron\AppData\Local\Temp\ipykernel_19864\534409118.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, start=start, end=end)[['Open','High','Low','Close','Volume']]
Failed to get ticker '..' reason: Expecting value: line 1 column 1 (char 0)
HTTP Error 500: 
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['..']: YFException("Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}")
C:\Users\aaron\AppData\Local\Temp\ipykernel_19864\534409118.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, start=start, end=end)[['Open','High','Low','Close','Volume']]
[*********************100%***********************]  1 of 1 completed
C:\Users\aaron\AppData\Local\Temp\ipykernel_19864\534409118.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.d

KeyboardInterrupt: 

In [ ]:
row = ticker_ranges.iloc[351]
start, end = row["min_date"], row["max_date"]
t = row['ticker']
print('##########################', t, start, end)
ticker_data = yf.Ticker(t).history(start=start, end=end)[['Open', 'High', 'Low', 'Close', 'Volume']]

### Download tickers in batches of 200

In [ ]:
ticker_ranges = ticker_ranges.sort_values("min_date").reset_index(drop=True)
asdfg
batch_size = 200
prices = {}

for i in range(0, len(ticker_ranges), batch_size):
    batch = ticker_ranges.iloc[i:i+batch_size]
    tickers = batch['ticker'].tolist()
    batch_start = batch['min_date'].min()
    batch_end   = batch['max_date'].max()
    
    try:
        data = yf.download(
            tickers,
            start=batch_start,
            end=batch_end
        )[['Open','High','Low','Close','Volume']]
        
        # store each ticker’s slice separately
        for t in tickers:
            if (t,) in data.columns:  # multiindex form
                prices[t] = data[t]
            else:  # single ticker form
                prices[t] = data
    except Exception as e:
        print(f"Batch failed {tickers[:5]}...: {e}")

In [1]:
import pickle

In [2]:
with open('saved_vars/500_trades_from_mid_2024.pkl', 'rb') as f:  # 'rb' = read binary
    data = pickle.load(f)


data

,transaction_date,trade_date,ticker,company_name,owner_name,Title,transaction_type,last_price,Qty,shares_held,Owned,Value,Owned_norm
0,2024-07-01 06:30:00,2024-06-27,STTK,"Shattuck Labs, Inc.",Brous Tyler,Dir,P - Purchase,3.89,8416.0,238088.0,4.0,32768.0,0.027345
1,2024-07-01 07:12:00,2024-06-25,EP,Empire Petroleum Corp,Mulacek Phil E,"Dir, 10%",P - Purchase,5.31,38543.0,6190976.0,1.0,204555.0,0.006908
2,2024-07-01 08:00:00,2024-06-27,TSHA,"Taysha Gene Therapies, Inc.",Manning Paul B,10%,P - Purchase,2.25,1333333.0,24851111.0,6.0,2999999.0,0.040736
3,2024-07-01 08:40:00,2024-06-28,FBIO,"Fortress Biotech, Inc.",Rosenwald Lindsay A Md,"Pres, CEO, COB, 10%",P - Purchase,1.64,20000.0,2893905.0,1.0,32724.0,0.006908
4,2024-07-01 09:17:00,2024-06-27,TURN,180 Degree Capital Corp. /ny/,Rendino Kevin,"COB, CEO",P - Purchase,3.69,15800.0,789299.0,2.0,58337.0,0.013767
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2024-07-30 18:25:00,2024-07-29,ZYXI,Zynex Inc,Disbrow Joshua R.,Dir,P - Purchase,8.32,7000.0,72000.0,11.0,58233.0,0.073412
496,2024-07-30 19:11:00,2024-07-26,LGF,Lions Gate Entertainment Corp /cn/,Liberty 77 Capital L.P.,10%,P - Purchase,8.50,385831.0,11055980.0,4.0,3278206.0,0.027345
497,2024-07-30 19:55:00,2024-07-30,MXL,"Maxlinear, Inc",Seendripu Kishore,"COB, Pres, CEO, 10%",P - Purchase,12.94,108303.0,4954999.0,2.0,1401592.0,0.013767
498,2024-07-31 08:20:00,2024-07-30,PFBX,Peoples Financial Corp /ms/,Stilwell Joseph,10%,P - Purchase,16.85,2000.0,614794.0,0.0,33700.0,0.000000
